In [1]:
import os
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

In [2]:
band_path = []
for i in range(1, 8):
    curr_band_path = "./Landsat/LC08_L2SP_022031_20200612_20200823_02_T1/"+"LC08_L2SP_022031_20200612_20200823_02_T1_SR_B"+str(i)+".TIF"
    band_path.append(curr_band_path)



In [3]:
band_path

['./Landsat/LC08_L2SP_022031_20200612_20200823_02_T1/LC08_L2SP_022031_20200612_20200823_02_T1_SR_B1.TIF',
 './Landsat/LC08_L2SP_022031_20200612_20200823_02_T1/LC08_L2SP_022031_20200612_20200823_02_T1_SR_B2.TIF',
 './Landsat/LC08_L2SP_022031_20200612_20200823_02_T1/LC08_L2SP_022031_20200612_20200823_02_T1_SR_B3.TIF',
 './Landsat/LC08_L2SP_022031_20200612_20200823_02_T1/LC08_L2SP_022031_20200612_20200823_02_T1_SR_B4.TIF',
 './Landsat/LC08_L2SP_022031_20200612_20200823_02_T1/LC08_L2SP_022031_20200612_20200823_02_T1_SR_B5.TIF',
 './Landsat/LC08_L2SP_022031_20200612_20200823_02_T1/LC08_L2SP_022031_20200612_20200823_02_T1_SR_B6.TIF',
 './Landsat/LC08_L2SP_022031_20200612_20200823_02_T1/LC08_L2SP_022031_20200612_20200823_02_T1_SR_B7.TIF']

In [4]:
band_data = []
for i in range(0, 7):
    curr_band_path = band_path[i]
    curr_band = rxr.open_rasterio(curr_band_path, masked=True).squeeze()
    band_data.append(curr_band)

In [5]:
band_data[0]

<xarray.DataArray (y: 7871, x: 7751)>
[61008121 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 4.032e+05 4.032e+05 ... 6.357e+05 6.357e+05
  * y            (y) float64 4.741e+06 4.741e+06 ... 4.505e+06 4.504e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [6]:
import pandas as pd
node = pd.read_csv("nodes.csv")

In [7]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:32616')

lat = node["lat"].tolist()
lon = node["lon"].tolist()
outx = []
outy = []
for i in range(0, len(lat)):
    x,y = transform(inProj,outProj,lon,lat)
    outx.append(x)
    outy.append(y)

/Users/carnivalbug/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/carnivalbug/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/Users/carnivalbug/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferre

In [8]:
outx = outx[0]
outy = outy[0]
node["x"]=0.0
node["y"]=0.0

In [9]:
for i in range(0, len(outx)):
    node.at[i,"x"] = outx[i]
    node.at[i,"y"] = outy[i]

In [10]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,y
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,4.636463e+06
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,4.634209e+06
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,4.641782e+06
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,4.628887e+06
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,4.635181e+06
...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06


In [11]:
node["612_1"]="-1"
node["612_2"]="-1"
node["612_3"]="-1"
node["612_4"]="-1"
node["612_5"]="-1"
node["612_6"]="-1"
node["612_7"]="-1"

In [12]:
for i in range(0, 126):
    try:
        x= node.at[i,"x"]
        y= node.at[i,"y"]
        curr_i = 0
        while(not (x>=int(band_data[0].x[curr_i]) and x<int(band_data[0].x[curr_i+1]))):
            curr_i = curr_i+1
        curr_j = 0
        while(not (y<=int(band_data[0].y[curr_j]) and y>int(band_data[0].y[curr_j+1]))):
            curr_j = curr_j+1
        node.at[i,"612_1"] = int(band_data[0][curr_j,curr_i])
        node.at[i,"612_2"] = int(band_data[1][curr_j,curr_i])
        node.at[i,"612_3"] = int(band_data[2][curr_j,curr_i])
        node.at[i,"612_4"] = int(band_data[3][curr_j,curr_i])
        node.at[i,"612_5"] = int(band_data[4][curr_j,curr_i])
        node.at[i,"612_6"] = int(band_data[5][curr_j,curr_i])
        node.at[i,"612_7"] = int(band_data[6][curr_j,curr_i])
    except:
        print(i)


In [13]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,y,612_1,612_2,612_3,612_4,612_5,612_6,612_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,4.636463e+06,12086,12515,14956,15272,20645,16908,14083
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,4.634209e+06,11429,11876,12916,13415,18929,16479,13987
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,4.641782e+06,8723,9451,11357,11114,19700,16069,13296
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,4.628887e+06,9652,9860,10790,10237,19764,13874,11396
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,4.635181e+06,10376,10775,11814,12286,13745,15093,14345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06,10585,11008,11864,12233,15061,14693,13214
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06,10585,11008,11864,12233,15061,14693,13214
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06,10585,11008,11864,12233,15061,14693,13214
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,4.631090e+06,10585,11008,11864,12233,15061,14693,13214


In [14]:
## start adding 7/5

In [15]:
band_path = []
for i in range(1, 8):
    curr_band_path = "./Landsat/LC08_L2SP_023031_20200705_20200913_02_T1/"+"LC08_L2SP_023031_20200705_20200913_02_T1_SR_B"+str(i)+".TIF"
    band_path.append(curr_band_path)

In [16]:
band_path

['./Landsat/LC08_L2SP_023031_20200705_20200913_02_T1/LC08_L2SP_023031_20200705_20200913_02_T1_SR_B1.TIF',
 './Landsat/LC08_L2SP_023031_20200705_20200913_02_T1/LC08_L2SP_023031_20200705_20200913_02_T1_SR_B2.TIF',
 './Landsat/LC08_L2SP_023031_20200705_20200913_02_T1/LC08_L2SP_023031_20200705_20200913_02_T1_SR_B3.TIF',
 './Landsat/LC08_L2SP_023031_20200705_20200913_02_T1/LC08_L2SP_023031_20200705_20200913_02_T1_SR_B4.TIF',
 './Landsat/LC08_L2SP_023031_20200705_20200913_02_T1/LC08_L2SP_023031_20200705_20200913_02_T1_SR_B5.TIF',
 './Landsat/LC08_L2SP_023031_20200705_20200913_02_T1/LC08_L2SP_023031_20200705_20200913_02_T1_SR_B6.TIF',
 './Landsat/LC08_L2SP_023031_20200705_20200913_02_T1/LC08_L2SP_023031_20200705_20200913_02_T1_SR_B7.TIF']

In [17]:
band_data = []
for i in range(0, 7):
    curr_band_path = band_path[i]
    curr_band = rxr.open_rasterio(curr_band_path, masked=True).squeeze()
    band_data.append(curr_band)

In [20]:
band_data[0]

<xarray.DataArray (y: 7951, x: 7841)>
[62343791 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 2.733e+05 2.733e+05 ... 5.085e+05 5.085e+05
  * y            (y) float64 4.743e+06 4.743e+06 ... 4.504e+06 4.504e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [21]:
node["705_1"]="-1"
node["705_2"]="-1"
node["705_3"]="-1"
node["705_4"]="-1"
node["705_5"]="-1"
node["705_6"]="-1"
node["705_7"]="-1"

In [23]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,612_5,612_6,612_7,705_1,705_2,705_3,705_4,705_5,705_6,705_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,20645,16908,14083,-1,-1,-1,-1,-1,-1,-1
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18929,16479,13987,-1,-1,-1,-1,-1,-1,-1
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,19700,16069,13296,-1,-1,-1,-1,-1,-1,-1
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,19764,13874,11396,-1,-1,-1,-1,-1,-1,-1
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,13745,15093,14345,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,15061,14693,13214,-1,-1,-1,-1,-1,-1,-1
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,15061,14693,13214,-1,-1,-1,-1,-1,-1,-1
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,15061,14693,13214,-1,-1,-1,-1,-1,-1,-1
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,15061,14693,13214,-1,-1,-1,-1,-1,-1,-1


In [24]:
for i in range(0, 126):
    try:
        x= node.at[i,"x"]
        y= node.at[i,"y"]
        curr_i = 0
        while(not (x>=int(band_data[0].x[curr_i]) and x<int(band_data[0].x[curr_i+1]))):
            curr_i = curr_i+1
        curr_j = 0
        while(not (y<=int(band_data[0].y[curr_j]) and y>int(band_data[0].y[curr_j+1]))):
            curr_j = curr_j+1
        node.at[i,"705_1"] = int(band_data[0][curr_j,curr_i])
        node.at[i,"705_2"] = int(band_data[1][curr_j,curr_i])
        node.at[i,"705_3"] = int(band_data[2][curr_j,curr_i])
        node.at[i,"705_4"] = int(band_data[3][curr_j,curr_i])
        node.at[i,"705_5"] = int(band_data[4][curr_j,curr_i])
        node.at[i,"705_6"] = int(band_data[5][curr_j,curr_i])
        node.at[i,"705_7"] = int(band_data[6][curr_j,curr_i])
    except:
        print(i)

In [25]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,612_5,612_6,612_7,705_1,705_2,705_3,705_4,705_5,705_6,705_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,20645,16908,14083,11271,11845,13981,14258,19563,16344,13487
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18929,16479,13987,10375,10830,12521,12857,18538,15920,13489
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,19700,16069,13296,7950,8507,10180,9987,18766,15356,12163
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,19764,13874,11396,9235,9345,10411,10120,17707,13367,11077
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,13745,15093,14345,10254,10715,11602,12130,13625,14682,13688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,15061,14693,13214,10084,10523,11555,11949,14662,14888,13502
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,15061,14693,13214,10084,10523,11555,11949,14662,14888,13502
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,15061,14693,13214,10084,10523,11555,11949,14662,14888,13502
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,15061,14693,13214,10084,10523,11555,11949,14662,14888,13502


In [26]:
## start adding 8/31

In [27]:
band_path = []
for i in range(1, 8):
    curr_band_path = "./Landsat/LC08_L2SP_022031_20200831_20200906_02_T1/"+"LC08_L2SP_022031_20200831_20200906_02_T1_SR_B"+str(i)+".TIF"
    band_path.append(curr_band_path)

In [28]:
band_path

['./Landsat/LC08_L2SP_022031_20200831_20200906_02_T1/LC08_L2SP_022031_20200831_20200906_02_T1_SR_B1.TIF',
 './Landsat/LC08_L2SP_022031_20200831_20200906_02_T1/LC08_L2SP_022031_20200831_20200906_02_T1_SR_B2.TIF',
 './Landsat/LC08_L2SP_022031_20200831_20200906_02_T1/LC08_L2SP_022031_20200831_20200906_02_T1_SR_B3.TIF',
 './Landsat/LC08_L2SP_022031_20200831_20200906_02_T1/LC08_L2SP_022031_20200831_20200906_02_T1_SR_B4.TIF',
 './Landsat/LC08_L2SP_022031_20200831_20200906_02_T1/LC08_L2SP_022031_20200831_20200906_02_T1_SR_B5.TIF',
 './Landsat/LC08_L2SP_022031_20200831_20200906_02_T1/LC08_L2SP_022031_20200831_20200906_02_T1_SR_B6.TIF',
 './Landsat/LC08_L2SP_022031_20200831_20200906_02_T1/LC08_L2SP_022031_20200831_20200906_02_T1_SR_B7.TIF']

In [29]:
band_data = []
for i in range(0, 7):
    curr_band_path = band_path[i]
    curr_band = rxr.open_rasterio(curr_band_path, masked=True).squeeze()
    band_data.append(curr_band)

In [30]:
band_data[0]

<xarray.DataArray (y: 7871, x: 7751)>
[61008121 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 4.035e+05 4.035e+05 4.036e+05 ... 6.36e+05 6.36e+05
  * y            (y) float64 4.741e+06 4.741e+06 ... 4.505e+06 4.504e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [31]:
node["831_1"]="-1"
node["831_2"]="-1"
node["831_3"]="-1"
node["831_4"]="-1"
node["831_5"]="-1"
node["831_6"]="-1"
node["831_7"]="-1"

In [32]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,705_5,705_6,705_7,831_1,831_2,831_3,831_4,831_5,831_6,831_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,19563,16344,13487,-1,-1,-1,-1,-1,-1,-1
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18538,15920,13489,-1,-1,-1,-1,-1,-1,-1
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,18766,15356,12163,-1,-1,-1,-1,-1,-1,-1
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,17707,13367,11077,-1,-1,-1,-1,-1,-1,-1
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,13625,14682,13688,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14662,14888,13502,-1,-1,-1,-1,-1,-1,-1
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14662,14888,13502,-1,-1,-1,-1,-1,-1,-1
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14662,14888,13502,-1,-1,-1,-1,-1,-1,-1
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14662,14888,13502,-1,-1,-1,-1,-1,-1,-1


In [33]:
for i in range(0, 126):
    try:
        x= node.at[i,"x"]
        y= node.at[i,"y"]
        curr_i = 0
        while(not (x>=int(band_data[0].x[curr_i]) and x<int(band_data[0].x[curr_i+1]))):
            curr_i = curr_i+1
        curr_j = 0
        while(not (y<=int(band_data[0].y[curr_j]) and y>int(band_data[0].y[curr_j+1]))):
            curr_j = curr_j+1
        node.at[i,"831_1"] = int(band_data[0][curr_j,curr_i])
        node.at[i,"831_2"] = int(band_data[1][curr_j,curr_i])
        node.at[i,"831_3"] = int(band_data[2][curr_j,curr_i])
        node.at[i,"831_4"] = int(band_data[3][curr_j,curr_i])
        node.at[i,"831_5"] = int(band_data[4][curr_j,curr_i])
        node.at[i,"831_6"] = int(band_data[5][curr_j,curr_i])
        node.at[i,"831_7"] = int(band_data[6][curr_j,curr_i])
    except:
        print(i)

In [34]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,705_5,705_6,705_7,831_1,831_2,831_3,831_4,831_5,831_6,831_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,19563,16344,13487,11894,12551,13558,14487,18749,15535,13386
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18538,15920,13489,10978,11395,12201,12509,18556,16313,13511
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,18766,15356,12163,8518,9058,10349,10426,17241,15975,12623
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,17707,13367,11077,9028,9293,10026,9789,16943,13199,10872
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,13625,14682,13688,10034,10583,11311,11842,13309,14203,13379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14662,14888,13502,9826,10550,11828,12436,14748,15143,13663
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14662,14888,13502,9826,10550,11828,12436,14748,15143,13663
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14662,14888,13502,9826,10550,11828,12436,14748,15143,13663
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14662,14888,13502,9826,10550,11828,12436,14748,15143,13663


In [35]:
## start adding 1009

In [36]:
band_path = []
for i in range(1, 8):
    curr_band_path = "./Landsat/LC08_L2SP_023031_20201009_20201016_02_T1/"+"LC08_L2SP_023031_20201009_20201016_02_T1_SR_B"+str(i)+".TIF"
    band_path.append(curr_band_path)

In [37]:
band_path

['./Landsat/LC08_L2SP_023031_20201009_20201016_02_T1/LC08_L2SP_023031_20201009_20201016_02_T1_SR_B1.TIF',
 './Landsat/LC08_L2SP_023031_20201009_20201016_02_T1/LC08_L2SP_023031_20201009_20201016_02_T1_SR_B2.TIF',
 './Landsat/LC08_L2SP_023031_20201009_20201016_02_T1/LC08_L2SP_023031_20201009_20201016_02_T1_SR_B3.TIF',
 './Landsat/LC08_L2SP_023031_20201009_20201016_02_T1/LC08_L2SP_023031_20201009_20201016_02_T1_SR_B4.TIF',
 './Landsat/LC08_L2SP_023031_20201009_20201016_02_T1/LC08_L2SP_023031_20201009_20201016_02_T1_SR_B5.TIF',
 './Landsat/LC08_L2SP_023031_20201009_20201016_02_T1/LC08_L2SP_023031_20201009_20201016_02_T1_SR_B6.TIF',
 './Landsat/LC08_L2SP_023031_20201009_20201016_02_T1/LC08_L2SP_023031_20201009_20201016_02_T1_SR_B7.TIF']

In [38]:
band_data = []
for i in range(0, 7):
    curr_band_path = band_path[i]
    curr_band = rxr.open_rasterio(curr_band_path, masked=True).squeeze()
    band_data.append(curr_band)

In [39]:
band_data[0]

<xarray.DataArray (y: 7951, x: 7841)>
[62343791 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 2.724e+05 2.724e+05 ... 5.076e+05 5.076e+05
  * y            (y) float64 4.743e+06 4.743e+06 ... 4.504e+06 4.504e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [40]:
node["1009_1"]="-1"
node["1009_2"]="-1"
node["1009_3"]="-1"
node["1009_4"]="-1"
node["1009_5"]="-1"
node["1009_6"]="-1"
node["1009_7"]="-1"

In [41]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,831_5,831_6,831_7,1009_1,1009_2,1009_3,1009_4,1009_5,1009_6,1009_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,18749,15535,13386,-1,-1,-1,-1,-1,-1,-1
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18556,16313,13511,-1,-1,-1,-1,-1,-1,-1
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,17241,15975,12623,-1,-1,-1,-1,-1,-1,-1
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,16943,13199,10872,-1,-1,-1,-1,-1,-1,-1
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,13309,14203,13379,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14748,15143,13663,-1,-1,-1,-1,-1,-1,-1
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14748,15143,13663,-1,-1,-1,-1,-1,-1,-1
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14748,15143,13663,-1,-1,-1,-1,-1,-1,-1
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14748,15143,13663,-1,-1,-1,-1,-1,-1,-1


In [42]:
for i in range(0, 126):
    try:
        x= node.at[i,"x"]
        y= node.at[i,"y"]
        curr_i = 0
        while(not (x>=int(band_data[0].x[curr_i]) and x<int(band_data[0].x[curr_i+1]))):
            curr_i = curr_i+1
        curr_j = 0
        while(not (y<=int(band_data[0].y[curr_j]) and y>int(band_data[0].y[curr_j+1]))):
            curr_j = curr_j+1
        node.at[i,"1009_1"] = int(band_data[0][curr_j,curr_i])
        node.at[i,"1009_2"] = int(band_data[1][curr_j,curr_i])
        node.at[i,"1009_3"] = int(band_data[2][curr_j,curr_i])
        node.at[i,"1009_4"] = int(band_data[3][curr_j,curr_i])
        node.at[i,"1009_5"] = int(band_data[4][curr_j,curr_i])
        node.at[i,"1009_6"] = int(band_data[5][curr_j,curr_i])
        node.at[i,"1009_7"] = int(band_data[6][curr_j,curr_i])
    except:
        print(i)

In [43]:
node

,node_id,project_id,vsn,address,lat,lon,description,start_timestamp,end_timestamp,x,...,831_5,831_6,831_7,1009_1,1009_2,1009_3,1009_4,1009_5,1009_6,1009_7
0,001e0610ba46,AoT_Chicago,004,State St & Jackson Blvd Chicago IL,41.878377,-87.627678,AoT Chicago (S) [C],2017/10/09 00:00:00,NaN,447918.216810,...,18749,15535,13386,10448,10986,11727,12020,13926,12507,11022
1,001e0610ba3b,AoT_Chicago,006,18th St & Lake Shore Dr Chicago IL,41.858136,-87.616055,AoT Chicago (S),2017/08/08 00:00:00,NaN,448866.515374,...,18556,16313,13511,11081,11367,11996,12118,17627,15330,12819
2,001e0610f02f,AoT_Chicago,00A,Lake Shore Drive & Fullerton Ave Chicago IL,41.926261,-87.630758,AoT Chicago (S) [CA],2018/05/07 00:00:00,NaN,447701.758685,...,17241,15975,12623,8113,8739,9974,9862,16989,14589,11775
3,001e0610ba8f,AoT_Chicago,00D,Cornell & 47th St Chicago IL,41.810342,-87.590228,AoT Chicago (S),2017/08/08 00:00:00,NaN,450973.743012,...,16943,13199,10872,9164,9379,10046,9850,14587,12197,10236
4,001e0610ba16,AoT_Chicago,010,Homan Ave & Roosevelt Rd Chicago IL,41.866349,-87.710543,AoT Chicago (S) [C],2018/07/18 00:00:00,NaN,441031.375397,...,13309,14203,13379,10052,10435,11459,11799,13299,13620,12597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,001e06118433,AoT_Chicago,10E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14748,15143,13663,10157,10696,11780,12105,14467,14500,13121
122,001e061183bf,AoT_Chicago,11A,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14748,15143,13663,10157,10696,11780,12105,14467,14500,13121
123,001e0611804d,AoT_Chicago,11E,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14748,15143,13663,10157,10696,11780,12105,14467,14500,13121
124,001e061182a2,AoT_Chicago,13B,ComEd Training Center,41.829806,-87.659467,AoT Chicago (S) [CP] {ComEd},2019/04/25 00:00:00,NaN,445239.085178,...,14748,15143,13663,10157,10696,11780,12105,14467,14500,13121


In [45]:
node.to_csv("node_with_band.csv")